## Overview

This notebook covers the ETL process for australian_users_items.json`, detailing each step from data extraction, transformation and loading. I will show the methods I used and the reasoning behind each phase.

In [1]:
import pandas as pd

## Data Extraction:

In [2]:
file_path = '../data/raw/australian_users_items.json'

with open(file_path, 'r', encoding='utf-8') as file:
    items = file.readlines()
    items = [eval(line.strip()) for line in items]

df_items=pd.DataFrame(items)

df_items_clean = df_items.copy()
df_items_clean

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Explode the items column: 

In [3]:

import pandas as pd

df_exploded = df_items_clean.explode('items').
df_items = df_exploded['items'].apply(pd.Series)

df_items_clean = pd.concat([df_exploded.drop('items', axis=1), df_items], axis=1)



## Data Transformation

1. Drop irrelevant columns 
2. Find Missing Values and drop them. They provided no relevant information for us since the exploded section was empty. 
3. change column name to game_id
4. Fixed data types
5. Check for duplicate rows.



Drop Columns

In [4]:

df_items_clean = df_items_clean.drop(columns=[0, 'user_url', 'playtime_2weeks', 'steam_id'])

In [5]:
missing_values = df_items_clean.isna().sum()
print(missing_values)

user_id                 0
items_count             0
item_id             16806
item_name           16806
playtime_forever    16806
dtype: int64


Finding rows with missing values

In [6]:
rows_with_missing_values = df_items_clean.isnull().any(axis=1)
df_items_clean[rows_with_missing_values].tail()

,user_id,items_count,item_id,item_name,playtime_forever
88298,76561198316380182,0,NaN,NaN,NaN
88299,76561198316970597,0,NaN,NaN,NaN
88300,76561198318100691,0,NaN,NaN,NaN
88307,XxLaughingJackClown77xX,0,NaN,NaN,NaN
88309,edward_tremethick,0,NaN,NaN,NaN


In [7]:
df_items_clean.dropna(inplace=True)

Change column name

In [8]:
new_name = {
    'item_id': 'game_id',
}

df_items_clean.rename(columns=new_name, inplace=True)

print(df_items_clean.columns)

Index(['user_id', 'items_count', 'game_id', 'item_name', 'playtime_forever'], dtype='object')


In [9]:
df_items_clean.head(10)

,user_id,items_count,game_id,item_name,playtime_forever
0,76561197970982479,277,10,Counter-Strike,6.0
0,76561197970982479,277,20,Team Fortress Classic,0.0
0,76561197970982479,277,30,Day of Defeat,7.0
0,76561197970982479,277,40,Deathmatch Classic,0.0
0,76561197970982479,277,50,Half-Life: Opposing Force,0.0
0,76561197970982479,277,60,Ricochet,0.0
0,76561197970982479,277,70,Half-Life,0.0
0,76561197970982479,277,130,Half-Life: Blue Shift,0.0
0,76561197970982479,277,300,Day of Defeat: Source,4733.0
0,76561197970982479,277,240,Counter-Strike: Source,1853.0


In [10]:
missing_values = df_items_clean.isna().sum()
print(missing_values)

user_id             0
items_count         0
game_id             0
item_name           0
playtime_forever    0
dtype: int64


Fix Data Types

In [11]:
df_items_clean['game_id'] = df_items_clean['game_id'].astype('Int64')

In [12]:
df_items_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 88308
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   game_id           Int64  
 3   item_name         object 
 4   playtime_forever  float64
dtypes: Int64(1), float64(1), int64(1), object(2)
memory usage: 240.8+ MB


## Loading/Saving the Data
1. Saved dataframes: `df_items_clean`
2. Saved the data in`.parquet` 
3. File Path: `'../data/processed/'`

In [13]:

save_path = '../data/processed/'

In [17]:
df_items_clean.to_parquet(save_path + 'df_items_clean.parquet')